# Estimation of Entropic risk measure
This notebook covers algorithms proposed in the JMLR paper (url) for computing sample-based estimates of two popular convex risk measures : Utility-based shortfall risk (UBSR) and Optimized Certainty Equivalent (OCE) risk. 

The goal of this exercise is to demonstrate correctness of the algorithms by comparing the solutions of the algorithms with the known, true risk values.

## UBSR Estimation

**Utility-based Shortfall Risk**: Given a random variable $X$, the UBSR of $X$ for the choice of loss function '$l$' and a risk threshold '$\lambda$' is defined as 

$SR_{l,\lambda}(X) = \inf \left\{ t \in \mathrm{R} | \mathbf{E}\left[ l(-X-t) \right] \le \lambda \right\}$

**UBSR Estimation Problem** : Given 'm' samples of $X$ : $\left\{Z_i\right\}_{i=1}^m$, estimate $SR_{l,\lambda}(X)$.

In [1]:
from dataclasses import dataclass, field
from typing import List
import numpy as np
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn.datasets as scd
import pyproximal
from tqdm import tqdm
%matplotlib notebook

import warnings
warnings.filterwarnings(action="ignore")

from distributions import Gaussian
from algorithms.base import Trainer, Portfolio_Optimization, RISK_TYPE
from algorithms.utility_based_shortfall_risk import SR, UBSR
from algorithms.optimized_certainty_equivalent import OCE, OCE_Risk
#from utils import clean_projection

SYMBOLS = ['circle','square','star']
PLOTLY_COLORS = px.colors.qualitative.Set2


### Distributions
A collection of distributions for which Value-at-Risk (quantile) and Conditional Value-at-Risk (superquantile) are known.

In [2]:
delta, epsilon = 1e-3, 1.
M, N = [10,100,1000], 1000

### Example 2: Entropic Risk
SR coincides with the entropic risk for the choice of loss function : $l(x) = e^{\beta x}, \beta > 0$ and threshold $\lambda = 1$.

In [3]:
BETA  = 0.5
l, threshold = (lambda x : np.exp(BETA * x)),  1.
ubsr = SR(l, threshold)

$$\textrm{For a Gaussian r.v. } X \sim \mathcal{N}(\mu,\sigma^2), \; \textrm{ we have } \mathrm{SR}_{l,\lambda}(X) = -\mu + \frac{\beta \sigma^2}{2}$$.

In [4]:
mu, std = -1, 4
distribution = Gaussian(mu, std)
true_SR = -mu + (BETA*std**2)/2 - np.log(threshold) / BETA

In [5]:
def simulate_entropic(ubsr, delta, M, distribution, true_SR):
    Errors = []
    for m in M:
        errors = []
        for _ in range(N):
            Z = distribution.sample(m)
            t = ubsr.UBSR_SB(Z, delta/np.sqrt(m))
            errors.append(t - true_SR)
        Errors.append(errors)
    return np.array(Errors)

In [6]:
Errors_np = simulate_entropic(ubsr, delta, M, distribution, true_SR)
group_labels = ['sample-size ='+str(m) for m in M]
fig = ff.create_distplot(Errors_np, group_labels, bin_size=.5,colors=PLOTLY_COLORS)
fig.update_layout(xaxis=dict(title=dict(text='Error $t_m - SR_{l,\lambda}(X)$')), 
                  yaxis=dict(title=dict(text='Empirical pdf')), width=600, height=400)
fig.show()

In [7]:
M_values = [10,50,100,200,300,400,500,650,800,1000,1200,1500,2000,2500]
Errors_np = simulate_entropic(ubsr, delta, M_values, distribution, true_SR)
MAE_means, MAE_std = np.abs(Errors_np).mean(axis=-1), np.abs(Errors_np).std(axis=-1)
MSE_means, MSE_std = (Errors_np**2).mean(axis=-1), (Errors_np**2).std(axis=-1)
x_values = np.sqrt(M_values)
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_values, y=MAE_means, name = 'q=1, MAE',
                         line=dict(color=PLOTLY_COLORS[0], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means+MAE_std, name = 'q=1, MAE + std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means-MAE_std, name = 'q=1, MAE - std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
# dash options include 'dash', 'dot', and 'dashdot'


fig.add_trace(go.Scatter(x=x_values, y=MSE_means, name='q=2, MSE',
                         line = dict(color=PLOTLY_COLORS[1], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means+MSE_std, name = 'q=2, MSE + std',
                         line = dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means-MSE_std, name = 'q=2, MSE - std',
                         line=dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend = False))

fig.update_xaxes(title_text='$\sqrt{m}$')
fig.update_yaxes(title_text='$|t_m - SR_{l,\lambda}(X)|^q$')

# fig.update_layout(
#         title=dict(
#             text='$\\text{MAE and MSE bounds on UBSR estimator } t_m$'
#         ),
#         width=600, height=400, 
# )
fig.update_layout(width=600, height=400)

fig.show()

### Example 2: Entropic Risk
- OCE coincides with the entropic risk for the choice of utility function : $u(x) = e^{\beta x}, \beta > 0$.

In [8]:
def simulate_entropic_oce(oce, delta, epsilon, M, distribution, true_OCE):
    Errors = []
    for m in M:
        errors = []
        for _ in range(N):
            Z = distribution.sample(m)
            _, s = oce.OCE_SAA(Z, delta/np.sqrt(m), epsilon)
            errors.append(s - true_OCE)
        Errors.append(errors)
    return np.array(Errors)

In [9]:
BETA, a, b  = 0.5, 1, 1 
u = (lambda x : (np.exp(BETA * x)-1)/BETA)
u_derivative = (lambda x : (BETA) * np.exp(BETA * x))
oce = OCE(u, u_derivative)
delta, epsilon = 1e-3, 1

In [10]:
mu,std = -1, 4
distribution = Gaussian(mu, std)
true_OCE = -mu + (BETA*std**2)/2
Errors_np = simulate_entropic_oce(oce, delta, epsilon, M, distribution, true_OCE)
group_labels = ['sample-size ='+str(m) for m in M]
fig = ff.create_distplot(Errors_np, group_labels, bin_size=.5, colors = PLOTLY_COLORS)
fig.update_layout(xaxis=dict(title=dict(text='Error $s_m - OCE_{u}(X)$')), 
                  yaxis=dict(title=dict(text='Empirical pdf')), width=600, height=400)
fig.show()

In [11]:
M_values = [10,50,100,200,300,400,500,650,800,1000,1200,1500,2000,2500]
Errors_np = simulate_entropic_oce(oce, delta, epsilon, M_values, distribution, true_OCE)
MAE_means, MAE_std = np.abs(Errors_np).mean(axis=-1), np.abs(Errors_np).std(axis=-1)
MSE_means, MSE_std = (Errors_np**2).mean(axis=-1), (Errors_np**2).std(axis=-1)
x_values = np.sqrt(M_values)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_values, y=MAE_means, name = 'q=1, MAE',
                         line=dict(color=PLOTLY_COLORS[0], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means+MAE_std, name = 'q=1, MAE + std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means-MAE_std, name = 'q=1, MAE - std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
# dash options include 'dash', 'dot', and 'dashdot'


fig.add_trace(go.Scatter(x=x_values, y=MSE_means, name='q=2, MSE',
                         line = dict(color=PLOTLY_COLORS[1], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means+MSE_std, name = 'q=2, MSE + std',
                         line = dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means-MSE_std, name = 'q=2, MSE - std',
                         line=dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend = False))

fig.update_xaxes(title_text='$\sqrt{m}$')
fig.update_yaxes(title_text='$|s_m - OCE_u(X)|^q$')
fig.update_layout(width=600, height=400)
fig.show()

### Entropic Risk Minimization

In [12]:
d = 5
projection = pyproximal.projection.SimplexProj(d, radius=1, maxiter = 2000, xtol = 1e-6)
clean_projection = lambda x : np.around(projection(x), 3)
rng = np.random.default_rng()
mu, sigma = rng.uniform(low=-5, high=5, size=d) , scd.make_spd_matrix(d)+scd.make_spd_matrix(d)+scd.make_spd_matrix(d)
BETA = 0.5

## True Value of Entropic Risk

In [13]:
from pypfopt.efficient_frontier import EfficientFrontier
ef = EfficientFrontier(mu, sigma)
weights = ef.max_quadratic_utility(BETA)
cleaned_weights = ef.clean_weights()
theta_star = np.array([y for x,y in cleaned_weights.items()])
ef.portfolio_performance(verbose=True)

Expected annual return: 484.3%
Annual volatility: 189.7%
Sharpe Ratio: 2.55


(np.float64(4.843222934977455),
 np.float64(1.8969483563740939),
 np.float64(2.553165413651531))

In [14]:
delta, epsilon, EPOCHS, N = 1e-3, 1., 500, 20

## Entropic Risk : via UBSR Optimization

In [16]:
l = (lambda x : np.exp(BETA * x))
threshold = 1
l_prime = (lambda x : BETA*np.exp(BETA * x))
sr = UBSR('Entropic risk', RISK_TYPE.UBSR, l, l_prime, delta, epsilon, Portfolio_Optimization, clean_projection)
entropic_risk = Trainer(sr, EPOCHS, -5, 5) 
entropic_risk.get_figure().show()

In [17]:
def run_ubsr(sr, epochs, mu, sigma):
    Theta = np.zeros((epochs+1, d))
    Theta[0] = np.ones(d)/d
    for k in range(1, epochs+1):
        Z_hat, Z = rng.multivariate_normal(mean = mu, cov = sigma , size = k), rng.multivariate_normal(mean = mu, cov = sigma , size = k)
        sr_k = sr.UBSR_SB(Portfolio_Optimization.F(Theta[k-1], Z_hat))
        l_prime_k = sr.loss_derivative(-Portfolio_Optimization.F(Theta[k-1], Z) - sr_k)
        grad = -(Portfolio_Optimization.grad_F(Theta[k-1], Z).T * l_prime_k).mean(axis=-1) / l_prime_k.mean()
        alpha = 1. / np.sqrt(k)
        Theta[k]  = clean_projection(Theta[k-1] - alpha * grad)
    return Theta

In [18]:
Errors = []
for n in range(N):
    Theta_UBSR = run_ubsr(sr, EPOCHS, mu, sigma)
    Errors.append(Theta_UBSR - theta_star)
Errors_L1, Errors_L2 = np.abs(Errors).mean(-1).T, np.power(Errors, 2).mean(-1).T
MAE_means, MAE_std = np.abs(Errors_L1).mean(axis=-1), np.abs(Errors_L1).std(axis=-1)
MSE_means, MSE_std = (Errors_L2).mean(axis=-1), (Errors_L2).std(axis=-1)
x_values = np.arange(EPOCHS+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_values, y=MAE_means, name = 'q=1, MAE',
                         line=dict(color=PLOTLY_COLORS[0], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means+MAE_std, name = 'q=1, MAE + std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means-MAE_std, name = 'q=1, MAE - std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
# dash options include 'dash', 'dot', and 'dashdot'


fig.add_trace(go.Scatter(x=x_values, y=MSE_means, name='q=2, MSE',
                         line = dict(color=PLOTLY_COLORS[1], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means+MSE_std, name = 'q=2, MSE + std',
                         line = dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means-MSE_std, name = 'q=2, MSE - std',
                         line=dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend = False))

fig.update_xaxes(title_text='$k$')
fig.update_yaxes(title_text='$\mathbf{E}\\left[\\lVert\\theta_k - \\theta^*\\rVert_q^q\\right]$')
fig.update_layout(width=600, height=400)
fig.show()

## Entropic Risk : via OCE Optimization

In [19]:
u, u_prime = (lambda x : (np.exp(BETA * x)- 1)/BETA), (lambda x : np.exp(BETA * x))  
oce = OCE_Risk('Entropic Risk (OCE)', RISK_TYPE.OCE, u, u_prime, delta, epsilon, Portfolio_Optimization, clean_projection)

In [20]:
def run_oce(oce, epochs, mu, sigma):
    Theta = np.zeros((epochs+1, d))
    Theta[0] = np.ones(d)/d
    for k in range(1, epochs+1):
        Z_hat, Z = rng.multivariate_normal(mean = mu, cov = sigma , size = k), rng.multivariate_normal(mean = mu, cov = sigma , size = k)
        sr_k = oce.OCE_SB(Portfolio_Optimization.F(Theta[k-1], Z_hat), oce.delta/np.sqrt(k), oce.epsilon)
        u_prime_k = oce.utility_derivative(-Portfolio_Optimization.F(Theta[k-1], Z) - sr_k)
        grad = -(Portfolio_Optimization.grad_F(Theta[k-1], Z).T * u_prime_k).mean(axis=-1)
        alpha = 1. / np.sqrt(k)
        Theta[k]  = clean_projection(Theta[k-1]  - alpha * grad)
    return Theta

In [21]:
Errors = []
for n in range(N):
    Theta_OCE = run_oce(oce, EPOCHS, mu, sigma)
    Errors.append(Theta_OCE - theta_star)
    
Errors_L1, Errors_L2 = np.abs(Errors).mean(-1).T, np.power(Errors, 2).mean(-1).T
MAE_means, MAE_std = np.abs(Errors_L1).mean(axis=-1), np.abs(Errors_L1).std(axis=-1)
MSE_means, MSE_std = (Errors_L2).mean(axis=-1), (Errors_L2).std(axis=-1)
x_values = np.arange(EPOCHS+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_values, y=MAE_means, name = 'q=1, MAE',
                         line=dict(color=PLOTLY_COLORS[0], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means+MAE_std, name = 'q=1, MAE + std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MAE_means-MAE_std, name = 'q=1, MAE - std',
                         line=dict(color=PLOTLY_COLORS[0], width=1, dash='dot'), mode='lines', showlegend =False))
# dash options include 'dash', 'dot', and 'dashdot'


fig.add_trace(go.Scatter(x=x_values, y=MSE_means, name='q=2, MSE',
                         line = dict(color=PLOTLY_COLORS[1], width=2, dash='dash'), mode='lines'))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means+MSE_std, name = 'q=2, MSE + std',
                         line = dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend =False))
fig.add_trace(go.Scatter(x=x_values, y=MSE_means-MSE_std, name = 'q=2, MSE - std',
                         line=dict(color=PLOTLY_COLORS[1], width=1, dash='dashdot'), mode='lines', showlegend = False))

fig.update_xaxes(title_text='$k$')
fig.update_yaxes(title_text='$\mathbf{E}\\left[\\lVert\\theta_k - \\theta^*\\rVert_q^q\\right]$')
fig.update_layout(width=600, height=400)
fig.show()